# Number of Non-Directed Interactions

In [24]:
from pip._internal import main as pip
try:
    import csv
except ImportError:
    pip(['install', 'csv'])
    import csv
try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except  ImportError:
    pip(['install', 'sparqlwrapper'])
    from SPARQLWrapper import SPARQLWrapper, JSON
    
import pandas
    
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
NonDirected = pandas.DataFrame(columns=['NonDirected Interactions Count'])
pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?NonDirectInt)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .
        MINUS { ?entity a wp:DirectedInteraction .}
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	NonDirected = NonDirected.append({
		'NonDirected Interactions Count': result["NonDirectInt"]["value"],
	 }, ignore_index=True)

In [25]:
NonDirected

,NonDirected Interactions Count
0,23281


# Total WP Interactions

In [26]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
totalWPInts = pandas.DataFrame(columns=['Total WP Interactions'])
pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?InteractionCount)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	totalWPInts = totalWPInts.append({
		'Total WP Interactions': result["InteractionCount"]["value"],
	 }, ignore_index=True)

formTotalWPInts = totalWPInts.loc[:, "Total WP Interactions"].apply(pandas.to_numeric, errors = 'ignore').sum()

In [27]:
totalWPInts

,Total WP Interactions
0,49526


# Count of Interaction Types

In [28]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntTypesCount = pandas.DataFrame(columns=['Interaction Type', 'Interaction Count'])

pathwayQuery = '''
      SELECT (substr(str(?o),41) as ?IntType) (COUNT(?o) as ?IntCount)
      WHERE {
        ?entity a ?o . 
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .
      } ORDER BY DESC(?IntCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntTypesCount = IntTypesCount.append({
		'Interaction Type': result["IntType"]["value"],
        'Interaction Count': result["IntCount"]["value"],
	 }, ignore_index=True)

In [29]:
IntTypesCount

,Interaction Type,Interaction Count
0,Interaction,49526
1,DirectedInteraction,26245
2,Catalysis,2386
3,Conversion,2295
4,Inhibition,2285
5,Binding,2069
6,ComplexBinding,2069
7,Stimulation,1319
8,TranscriptionTranslation,508


# Interaction Count with Unspecified Type

In [30]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
Nonspecific = pandas.DataFrame(columns=['NonSpecified Interaction Type Count'])

pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?NonSpecific)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> 
        MINUS { 
        { ?entity a wp:DirectedInteraction .  }
          UNION
        { ?entity a wp:ComplexBinding .  }
          UNION
        { ?entity a wp:Binding .  }
        }
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	Nonspecific = Nonspecific.append({
		'NonSpecified Interaction Type Count': result["NonSpecific"]["value"],
	 }, ignore_index=True)

In [31]:
Nonspecific

,NonSpecified Interaction Type Count
0,21212


# Participants for Interactions: 

In [32]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
parts4Ints = pandas.DataFrame(columns=['Interaction', 'Interaction Type', 'Interaction Participants'])


pathwayQuery = '''

    SELECT DISTINCT (substr(str(?interaction),37 ) as ?interactionId ) (substr(str(?intType), 41 ) as ?types) (GROUP_CONCAT(DISTINCT substr(str(?partType),41), ", ") AS ?participants)  WHERE {
       ?interaction a wp:Interaction ;
           dcterms:isPartOf ?pathway ;
           a ?intType .
       ?interaction wp:participants ?participant .
       ?participant a ?partType .
       ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .

       FILTER (?partType != wp:DataNode)
       FILTER (?intType != wp:Interaction)
    } 
    LIMIT 20
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    parts4Ints = parts4Ints.append({
        'Interaction': result["interactionId"]["value"],
        'Interaction Type': result["types"]["value"],
        'Interaction Participants': result["participants"]["value"],
       
    }, ignore_index=True)

In [33]:
parts4Ints

,Interaction,Interaction Type,Interaction Participants
0,WP710_r95580/ComplexBinding/ec35d,ComplexBinding,"Complex, GeneProduct"
1,WP3257_r98244/WP/Interaction/b31e3,DirectedInteraction,Metabolite
2,WP1984_r103002/WP/Interaction/idb809d168,Stimulation,"Metabolite, Protein"
3,WP702_r102200/ComplexBinding/c66be,Binding,"Complex, GeneProduct"
4,WP4290_r97977/WP/Interaction/id3f69820b,DirectedInteraction,"Conversion, DirectedInteraction, GeneProduct, ..."
5,WP306_r97459/WP/Interaction/c9742,DirectedInteraction,"GeneProduct, Protein"
6,WP179_r103430/ComplexBinding/bc18b,ComplexBinding,"Complex, GeneProduct, Protein"
7,WP3641_r102505/WP/Interaction/idd6bc3e2a,DirectedInteraction,"GeneProduct, Metabolite"
8,WP3888_r102004/WP/Interaction/d1c10_1,DirectedInteraction,"GeneProduct, Protein"
9,WP3182_r80761/WP/Interaction/e8135,DirectedInteraction,"DirectedInteraction, Interaction, Metabolite"


# Interaction counts by participants

In [34]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntCounts4Parts = pandas.DataFrame(columns=['Interaction Participants', 'Interaction Count'])

pathwayQuery = '''

    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT ?participants (COUNT(?interaction) AS ?count) WHERE {
      SELECT ?interaction (GROUP_CONCAT(substr(str(?partType),41), ", ") AS ?participants) WHERE {
        ?interaction a wp:Interaction . 
        ?interaction wp:participants ?participant . 
        ?participant a ?partType .
        ?interaction dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .
        FILTER (?partType != wp:DataNode)
      } GROUP BY ?interaction
    } GROUP BY ?participants
      ORDER BY DESC(?count)
      
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    IntCounts4Parts = IntCounts4Parts.append({
        'Interaction Participants': result["participants"]["value"],
        'Interaction Count': result["count"]["value"],
       
    }, ignore_index=True)

In [35]:
IntCounts4Parts

,Interaction Participants,Interaction Count
0,"Metabolite, Metabolite",6192
1,"GeneProduct, GeneProduct",6018
2,"Protein, GeneProduct, Protein, GeneProduct",2760
3,Metabolite,1419
4,GeneProduct,1285
5,"GeneProduct, Protein, GeneProduct",1027
6,"Protein, GeneProduct",943
7,"GeneProduct, DirectedInteraction, Interaction",805
8,"Metabolite, GeneProduct",730
9,"Protein, GeneProduct, GeneProduct",630


# Identifier IDs by data source

In [37]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntIDs = pandas.DataFrame(columns=['Database Source', 'Interactions Counted'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT Distinct (count(?Interaction) as ?IntCount) ?database 
       WHERE {
        ?Interaction a gpml:Interaction ;
            gpml:xrefDataSource ?database .
        ?Interaction dcterms:isPartOf ?pathway .
        ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .
          
       } ORDER BY DESC(?IntCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntIDs = IntIDs.append({
		'Database Source': result["database"]["value"],
        'Interactions Counted': result["IntCount"]["value"],
	 }, ignore_index=True)

In [38]:
IntIDs

,Database Source,Interactions Counted
0,Reactome,990
1,Uniprot-TrEMBL,213
2,Rhea,123
3,KEGG Reaction,97
4,KEGG Pathway,28
5,WikiPathways,24
6,pato,8
7,KEGG Compound,7
8,WormBase,6
9,ChEBI,6


# GPML RDF Interactions WITHOUT a WP RDF equivalent

In [39]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
noWPINT = pandas.DataFrame(columns=['Interactions Counted'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT (COUNT(?Interaction) AS ?count) 
    WHERE {

          ?Interaction a gpml:Interaction .
          ?Interaction dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .
          MINUS {?wpInteraction wp:isAbout ?Interaction}

        } 
          ORDER BY DESC(?count)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	noWPINT = noWPINT.append({
        'Interactions Counted': result["count"]["value"],
	 }, ignore_index=True)

In [40]:
noWPINT

,Interactions Counted
0,46323


# GPML RDF Interactions WITH a WP RDF equivalent and type

In [41]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
wpIntTypes = pandas.DataFrame(columns=['Interaction Type', 'Interactions Count'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT (substr(STR(?IntType),41) AS ?type)  (COUNT(?InteractionWP) AS ?count) 
    WHERE {

        ?Interaction a gpml:Interaction .
        #    gpml:xrefDataSource ?provenence .
        ?InteractionWP  wp:isAbout ?Interaction ;
            a ?IntType .
        ?Interaction dcterms:isPartOf ?pathway .
        ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .

        } GROUP BY ?IntType
          ORDER BY DESC(?count)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	wpIntTypes = wpIntTypes.append({
		'Interaction Type': result["type"]["value"],
        'Interactions Count': result["count"]["value"],
	 }, ignore_index=True)

In [42]:
wpIntTypes

,Interaction Type,Interactions Count
0,Interaction,50395
1,DirectedInteraction,28965
2,Conversion,2742
3,Catalysis,2386
4,Inhibition,2296
5,Stimulation,1327
6,TranscriptionTranslation,542


# Coverage % for GPML Interaction with Equivalent WP RDF out of Total GPML Interactions

In [43]:
IntTable = wpIntTypes.loc[:,"Interactions Count"]
wpInteractionType = IntTable.apply(pandas.to_numeric, errors='ignore').sum() 
noWPIntTable = noWPINT.loc[:,'Interactions Counted']
noWPIntIneger = noWPIntTable.apply(pandas.to_numeric, errors = 'ignore').sum()
totalWPInt = noWPIntIneger + wpInteractionType
coverageInt = (wpInteractionType / totalWPInt) * 100
formPercentInt = str(round(coverageInt, 3)) + "% of GPML Interactions with an WP equivalent"

In [44]:
formPercentInt

'65.681% of GPML Interactions with an WP equivalent'

In [45]:
percentWoInt = 100 - coverageInt
str(round(percentWoInt, 3)) + "% of GPML Interactions without a WP equivalent"

'34.319% of GPML Interactions without a WP equivalent'

# Total GPML Interactions

In [46]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
GPMLCountTotal = pandas.DataFrame(columns=['Interactions Count'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT DISTINCT (COUNT(?Interaction) AS ?count) 
    WHERE {

          ?Interaction a gpml:Interaction .
          ?Interaction dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .
          
        } 
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	GPMLCountTotal = GPMLCountTotal.append({
        'Interactions Count': result["count"]["value"],
	 }, ignore_index=True)

GPMLCountTotalInt = GPMLCountTotal.loc[:,"Interactions Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 

In [47]:
GPMLCountTotalInt

95998

# % of WP RDF Interactions that are of unspecified type

In [48]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntTotalCount = pandas.DataFrame(columns=['Interaction Count'])

pathwayQuery = '''
      SELECT DISTINCT (COUNT(?genInteraction) as ?IntCount)
      WHERE {
        ?genInteraction a wp:Interaction .
        ?genInteraction dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> .
      }
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntTotalCount = IntTotalCount.append({
        'Interaction Count': result["IntCount"]["value"],
	 }, ignore_index=True)

NonSpecificTable = Nonspecific.loc[:,"NonSpecified Interaction Type Count"]
NonSpecificInteger = NonSpecificTable.apply(pandas.to_numeric, errors = 'ignore').sum()
TotalInteractions = IntTotalCount.loc[:, "Interaction Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 
percentNonSpecific = (NonSpecificInteger / TotalInteractions) * 100
formPercentNonSpecific = str(round(percentNonSpecific,3)) + '% of WP RDF Interaction with Unspecified Interaction Type'

In [49]:
formPercentNonSpecific

'42.83% of WP RDF Interaction with Unspecified Interaction Type'

In [50]:
percentWSpecificType = 100 - percentNonSpecific
str(round(percentWSpecificType, 3)) + "% of WP RDF Interactions with a specific WP RDF Interaction type"

'57.17% of WP RDF Interactions with a specific WP RDF Interaction type'

# % of NonDirected Interactions

In [51]:
NonDirectedInteger = NonDirected.loc[:, "NonDirected Interactions Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 
percentNonDirected = (NonDirectedInteger / TotalInteractions) * 100
#check this number
formPercentNonDirected = str(round(percentNonDirected, 3)) + '% of Non-directed interaction in WP RDF'

In [52]:
formPercentNonDirected

'47.008% of Non-directed interaction in WP RDF'

In [53]:
ConversionCount = int(IntTypesCount.iloc[0]["Interaction Count"])
BindingCount = int(IntTypesCount.iloc[1]["Interaction Count"])
InteractionWPRDFCount = int(IntTypesCount.iloc[2]["Interaction Count"])
DirectedWPRDFCount = int(IntTypesCount.iloc[3]["Interaction Count"])
CatalysisCount = int(IntTypesCount.iloc[4]["Interaction Count"])
TranscrTranslaCount = int(IntTypesCount.iloc[5]["Interaction Count"])
complexCount = int(IntTypesCount.iloc[6]["Interaction Count"])
InhibitionCount = int(IntTypesCount.iloc[7]["Interaction Count"])
stimulationCount = int(IntTypesCount.iloc[8]["Interaction Count"])

ConversionCount, BindingCount, InteractionWPRDFCount, DirectedWPRDFCount, CatalysisCount, TranscrTranslaCount, complexCount, InhibitionCount, stimulationCount

(49526, 26245, 2386, 2295, 2285, 2069, 2069, 1319, 508)

In [54]:
IntTypesCount

,Interaction Type,Interaction Count
0,Interaction,49526
1,DirectedInteraction,26245
2,Catalysis,2386
3,Conversion,2295
4,Inhibition,2285
5,Binding,2069
6,ComplexBinding,2069
7,Stimulation,1319
8,TranscriptionTranslation,508


In [55]:
#noWPIntIneger without eq
#GPMLCountTotalInt GPML Total
percentComplex = (complexCount / GPMLCountTotalInt ) * 100
formpercentComplex = str(round(percentComplex, 3)) + '% of complex interactions out of all GPML Interactions'
percentComplexWPRDF = (complexCount / formTotalWPInts) * 100
formpercentComplexWPRDF = str(round(percentComplexWPRDF, 3)) + '% of complex interactions out of all WP Interactions'
formpercentComplex,  formpercentComplexWPRDF

('2.155% of complex interactions out of all GPML Interactions',
 '4.178% of complex interactions out of all WP Interactions')

In [56]:
#complexCount = int(IntTypesCount.iloc[6]["Interaction Count"])
PercentConversionCount = (ConversionCount / formTotalWPInts) * 100
formPercentConversionCount = str(round(PercentConversionCount, 3)) + '% of conversion interactions out of all WP Interactions'
PercentBindingCount = (BindingCount / formTotalWPInts) * 100
formPercentBindingCount = str(round(PercentBindingCount, 3)) + '% of binding interactions out of all WP Interactions'
formPercentConversionCount, formPercentBindingCount

('100.0% of conversion interactions out of all WP Interactions',
 '52.992% of binding interactions out of all WP Interactions')

In [57]:
IntTypesCount

,Interaction Type,Interaction Count
0,Interaction,49526
1,DirectedInteraction,26245
2,Catalysis,2386
3,Conversion,2295
4,Inhibition,2285
5,Binding,2069
6,ComplexBinding,2069
7,Stimulation,1319
8,TranscriptionTranslation,508


In [58]:
totalWPInts, GPMLCountTotalInt

(  Total WP Interactions
 0                 49526, 95998)